## Big Data Analytics Team Challenge
sponsers: CDC, Booz Allen Hamilton,Emory University’s Rollins School of Public Health,and Task
Force for Global Health 

This challenge is about collaborating with expert and novice data scientists to develop an accurate, machine-learning algorithm for the rapid identification of unusual strains of Bird Flu. 

Influenza virus continues to be a major public health risk with new outbreaks and pandemics being identified and recorded every year. Among influenza strains, H5N1 Avian influenza (aka Bird flu) represents a major threat due to its high pathogenicity and mortality rates and lack of effective vaccines.

With current approaches, isolation and characterization of H5N1 influenza strains using sequencing techniques can be performed rapidly, but risk assessment of these data using conventional approaches can be challenging. New technologies and analytical tools recently developed can now be tailored to provide risk assessment analysis of genomic data.

In this challenge, your team will be given a large genomic data set.  Your team will be asked to devise machine-learning methods such as neural networks, clustering analysis, etc. to develop an algorithm that quickly identifies variations in H5N1 Avian Influenza strains that may be of interest for further, detailed investigation.  

Scoring Criteria:

In general, each team’s solution will be judged on its solution, design and innovation.  Specifically, each team’s solution will be judged based on the following criteria:

•      Accuracy of solution to data set #1 = 30%

•      Accuracy of solution to data set #2 = 30%

•      Innovation of solution = 20%

•      Providing an example of how your teams solutions could address a specific scenario (showing your work, process, and solution = 20%

It is understood that instructions for implementation of algorithms and solutions will be provided to the team prior to receiving any award

In [1]:
import pandas as pd
import numpy as np

In [2]:
location='/home/ds/notebooks/Data/FLU_Challenge_vietnam.csv'
df=pd.read_csv(location)
df.head()

,ID,seqs
0,0,agcaaaagcaggggtccaatctgtcaaaatggaaaaaatagtgctt...
1,0,atggagaaaatagtgcttcttcttgcaatagttagccttgttaaaa...
2,0,atggagaaaatagtgcttcttcttgcaatggtcagccttgttaaaa...
3,0,atggagaaaatagtgcttcttcttgcaatagtcagccttgttaaaa...
4,0,atggagaaaatagtgcttcttcttgcaatagtcagccttgttaaaa...


In [3]:
x,y=df.shape
#print '%s'%x
print "The Vietnam Data Sets has {0} observations and {1} features".format(x,y)
print "There are {0} nulls in the ID column.".format(df['ID'].isnull().values.ravel().sum())

The Vietnam Data Sets has 630 observations and 2 features
There are 115 nulls in the ID column.


Genetics is not my domain, but I do realize there is a lot of information packed within that gene sequence. I do know the individual letters represent nucleotides. Might want to dust off the old biology book from high school. So what are all the different types of individual letters do we have as of now?

In [5]:
s=pd.Series(df['seqs'])[0]
l2=['adenine','cytosine','thymine','guanine']
for x,y in zip([ x for y,x in enumerate(iter(set(s))) ],l2):
    print '{0}-{1}'.format(x.upper(),y.upper())

A-ADENINE
C-CYTOSINE
T-THYMINE
G-GUANINE


# Nucleotides:
A stands for Adenine, C for cytonsine, T for thymine, and G for guanine are the meaning of each letter. Looking at this problem from a more NLP background. I am building up an alphabet and now its time to construct a vocabulary. 

In [6]:
sequences = df['seqs'].tolist()

characters=[]
for x in sequences:
    characters.append(list(x))

letters=[]
for x in characters:
    letters.append(set(x))

In [7]:
def sequence(row,n):
    s=row['seqs']
    k=len(s)-2*n
    klist = []
    for i in range(k):
        kmer=s[i:i+n]
        klist.append(kmer)
        #if not(kmer in klist) and (kmer in s[i+n:]):
        knmers = ' '.join(klist)   
    return knmers

In [8]:
def length(row):
    x = len(row['seqs'])
    return x

df['length']=df.apply (lambda row: length (row),axis=1)

In [9]:
df['sequence']=df.apply (lambda row: sequence(row,3),axis=1)

In [10]:
df[:1]

,ID,seqs,length,sequence
0,0,agcaaaagcaggggtccaatctgtcaaaatggaaaaaatagtgctt...,1779,agc gca caa aaa aaa aag agc gca cag agg ggg gg...


In [159]:
import re
print len(re.findall(r' ', df['sequence'][0]))

1772


In [18]:
from collections import defaultdict
d = defaultdict(int)
for x in df['sequence'][0].split(' '):
    d[x] += 1

In [19]:
d

defaultdict(int,
            {'aaa': 79,
             'aac': 33,
             'aag': 49,
             'aat': 67,
             'aca': 36,
             'acc': 26,
             'acg': 10,
             'act': 26,
             'aga': 66,
             'agc': 24,
             'agg': 27,
             'agt': 29,
             'ata': 36,
             'atc': 24,
             'atg': 55,
             'att': 37,
             'caa': 60,
             'cac': 19,
             'cag': 35,
             'cat': 27,
             'cca': 36,
             'ccc': 11,
             'ccg': 3,
             'cct': 18,
             'cga': 13,
             'cgc': 2,
             'cgg': 9,
             'cgt': 3,
             'cta': 18,
             'ctc': 25,
             'ctg': 19,
             'ctt': 19,
             'gaa': 59,
             'gac': 22,
             'gag': 36,
             'gat': 33,
             'gca': 31,
             'gcc': 8,
             'gcg': 6,
             'gct': 16,
             'gga': 42,
     

In [ ]:
df['sequence']=df.apply (lambda row: sequence(row,3),axis=1)

In [20]:
knmers=[]
def sequence_unique(row,n):
    s=row['seqs']
    k=len(s)-2*n
    klist = []
    for i in range(k):
        kmer=s[i:i+n]
        if not(kmer in klist) and (kmer in s[i+n:]):
            klist.append(kmer)
            knmers = ','.join(klist)   
    return knmers
totalvocab_tokenized = []
for index,row in df.iterrows():
    #allwords_stemmed = sequence_unique(i) #for each item in 'synopses', tokenize/stem
    #totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = sequence_unique(row,3)
    totalvocab_tokenized.extend(allwords_tokenized)

In [21]:
allwords_tokenized
#totalvocab_tokenized[:10]

'tct,ctg,tgt,gtc,tca,caa,aaa,aat,atg,tgg,gga,gag,aga,gaa,ata,tag,agt,gtg,tgc,gct,ctt,ttc,ttt,ttg,gcg,cga,gat,cag,gtt,tta,taa,aag,tga,atc,att,gca,cat,ggt,tac,acc,cca,aac,aca,act,ctc,tcg,gac,agc,agg,cac,acg,cgt,gcc,ccc,cgg,ggg,cta,cct,gta,ggc,tcc,ccg,tat,cgc'

In [22]:
vocab_frame = pd.DataFrame({'sequence': allwords_tokenized}, index = df['length'])
print 'there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame'

there are 630 items in vocab_frame


In [26]:
allwords_tokenized

'tct,ctg,tgt,gtc,tca,caa,aaa,aat,atg,tgg,gga,gag,aga,gaa,ata,tag,agt,gtg,tgc,gct,ctt,ttc,ttt,ttg,gcg,cga,gat,cag,gtt,tta,taa,aag,tga,atc,att,gca,cat,ggt,tac,acc,cca,aac,aca,act,ctc,tcg,gac,agc,agg,cac,acg,cgt,gcc,ccc,cgg,ggg,cta,cct,gta,ggc,tcc,ccg,tat,cgc'

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2,
                                 use_idf=True, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(df['sequence']) #fit the vectorizer to synopses

print(tfidf_matrix.shape)

CPU times: user 3.57 s, sys: 32.8 ms, total: 3.6 s
Wall time: 3.61 s
(630, 179)


In [24]:
terms = tfidf_vectorizer.get_feature_names()

In [25]:
len(terms)

179

In [26]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)
print
print

In [34]:
from sklearn.cluster import KMeans

num_clusters = 2

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

CPU times: user 203 ms, sys: 0 ns, total: 203 ms
Wall time: 203 ms


In [35]:
genes = { 'ID': df['ID'], 'sequence': df['sequence'],'length':df['length'] ,'cluster': clusters}

frame = pd.DataFrame(genes, index = [clusters] , columns = ['ID', 'sequence', 'cluster','length'])

In [36]:
frame['cluster'].value_counts()

1    481
0    149
Name: cluster, dtype: int64

In [37]:
grouped = frame['length'].groupby(frame['cluster']) #groupby cluster for aggregation purposes

grouped.mean()

cluster
0    1779
1    1704
Name: length, dtype: int64

In [47]:
from __future__ import print_function

print("Top terms per cluster:")
print()
#sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
        print(' %s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=',')
    print() #add whitespace
    print() #add whitespace
    
    print("Cluster %d titles:" % i, end='')
    for title in frame.ix[i]['title'].values.tolist():
        print(' %s,' % title, end='')
    print() #add whitespace
    print() #add whitespace
    
print()
print()

Top terms per cluster:

Cluster 0 words:

AttributeError: 'float' object has no attribute 'encode'

In [41]:
df[(df['ID']==1)]

,ID,seqs,length,sequence
467,1,aaatggagaaaatagtgcttctttttgcgatagtcagtcttgttaa...,1731,aaa aat atg tgg gga gag aga gaa aaa aaa aat at...
468,1,atggagaaaatagtgcttctttttgcgatagtcagtcttgttaaaa...,1732,atg tgg gga gag aga gaa aaa aaa aat ata tag ag...
469,1,atggagaaaatagtgcttctttttgcgatagtcagtcttgttaaaa...,1728,atg tgg gga gag aga gaa aaa aaa aat ata tag ag...
470,1,atggagaaaatagtgcttctttttgcgatagtcagtcttgttaaaa...,1728,atg tgg gga gag aga gaa aaa aaa aat ata tag ag...
471,1,atggagagaatagtgcttctttttgcaatagtcagtcttgttaaaa...,1707,atg tgg gga gag aga gag aga gaa aat ata tag ag...
472,1,atggagaaaatagtgcttctttttgcgatagtcagtcttgttaaaa...,1731,atg tgg gga gag aga gaa aaa aaa aat ata tag ag...
473,1,atggagaaaatagtgcttctttttgcgatagtcagtcttgttaaaa...,1731,atg tgg gga gag aga gaa aaa aaa aat ata tag ag...
474,1,atggagaaaatagtgcttctttttgcgatagtcagtcttgttaaaa...,1731,atg tgg gga gag aga gaa aaa aaa aat ata tag ag...
475,1,atggagaaaatagtgcttctttttgcgatagtcagtcttgttaaaa...,1728,atg tgg gga gag aga gaa aaa aaa aat ata tag ag...
476,1,tctgtcaaaatggagaaaatagtgcttcttcttgcaatagtcaacc...,1733,tct ctg tgt gtc tca caa aaa aaa aat atg tgg gg...
